# Get Data Ready

In [1]:
!pip install gdown

In [1]:

#https://drive.google.com/file/d/1OTwtZadt0VpdbK3oo7ap3FIANzzdBcx6/view?usp=sharing, 
!gdown --id 1OTwtZadt0VpdbK3oo7ap3FIANzzdBcx6
#https://drive.google.com/file/d/1Wbc1RDWpw74GaX7W7l8Kl2ymNqoj87uV/view?usp=sharing
!gdown --id 1Wbc1RDWpw74GaX7W7l8Kl2ymNqoj87uV

E:\Program_Files\Anaconda\envs\685_final\lib\site-packages\gdown\cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1OTwtZadt0VpdbK3oo7ap3FIANzzdBcx6
To: e:\UMASS\685_e\github\Zihao_branch\bilimemenet\6_cates\roberta_chinese\danmu_text_6_cates.pkl

  0%|          | 0.00/3.39M [00:00<?, ?B/s]
 31%|███       | 1.05M/3.39M [00:00<00:00, 5.41MB/s]
 62%|██████▏   | 2.10M/3.39M [00:00<00:00, 5.54MB/s]
 93%|█████████▎| 3.15M/3.39M [00:00<00:00, 5.58MB/s]
100%|██████████| 3.39M/3.39M [00:00<00:00, 5.56MB/s]
E:\Program_Files\Anaconda\envs\685_final\lib\site-packages\gdown\cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Wbc1RDWpw74GaX7W7l

In [2]:
import pickle

token_file = open("danmu_text_6_cates.pkl", "rb")
label_file = open("danmu_label_6_cates.pkl", "rb")

tokens = pickle.load(token_file)
labels = pickle.load(label_file)

print(len(tokens))
print(len(labels))

149981
149981


In [4]:
import numpy as np

label_distribution = {}
for i in labels:
    if i in label_distribution:
        label_distribution[i] += 1
    else:
        label_distribution[i] = 1

In [5]:
print(label_distribution)
len(label_distribution)

{1: 16438, 5: 38976, 6: 9903, 2: 41752, 3: 13504, 4: 29408}


6

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.DataFrame({
    'text': tokens,
    'label': labels
})

train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.1)

In [7]:
#Debugging
train_df = train_df[train_df['text'] != 'nan']
test_df = test_df[test_df['text'] != 'nan']
val_df = val_df[val_df['text'] != 'nan']


In [8]:
train_df.reset_index(drop=True)
test_df.reset_index(drop=True)
val_df.reset_index(drop=True)

,text,label
0,双厨爆炸,1
1,这是什么阴乐,1
2,钓鱼是吧,6
3,欲看又止,4
4,醉好吃的,5
...,...,...
11994,同款清洁,4
11995,不你屑屑的,4
11996,国足yyds,3
11997,已经退坑,2


In [11]:
#save to csv
import os
train_df = train_df.dropna()
test_df = test_df.dropna()
val_df = val_df.dropna()

if not os.path.exists('data_csv'):
    os.makedirs('data_csv')

train_df.to_csv('data_csv/train.csv', index = False)
test_df.to_csv('data_csv/test.csv', index = False)
val_df.to_csv('data_csv/val.csv', index = False)

# Preprocessing

In [12]:
import datasets
train = datasets.Dataset.from_pandas(train_df, preserve_index=False)
val = datasets.Dataset.from_pandas(val_df, preserve_index=False)
test = datasets.Dataset.from_pandas(test_df, preserve_index=False)

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("uer/chinese_roberta_L-12_H-768")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

Downloading: 100%|██████████| 468/468 [00:00<00:00, 235kB/s]
Downloading: 100%|██████████| 107k/107k [00:00<00:00, 2.15MB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 56.0kB/s]


In [14]:
#Tokenization

tokenized_train =  train.map(preprocess_function, batched=True)
tokenized_val =  val.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

100%|██████████| 30/30 [00:00<00:00, 46.51ba/s]


In [15]:
# For batching and padding
import transformers 

data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
#!conda install pytorch torchvision torchaudio cpuonly -c pytorch

# Model

In [39]:
model = transformers.AutoModelForMaskedLM.from_pretrained("uer/chinese_roberta_L-12_H-768",
                                                        num_labels = len(label_distribution),
                                                        output_attentions = False, 
                                                        output_hidden_states = False,)



loading configuration file https://huggingface.co/uer/chinese_roberta_L-12_H-768/resolve/main/config.json from cache at C:\Users\leima/.cache\huggingface\transformers\0581d850743cad42501488567cceb8dc9ce50d9f05ad632d273b4389b2f52f68.042085124aedc502028136283b7bf9a169a238009bd6c309f049b249216061a2
Model config BertConfig {
  "_name_or_path": "uer/chinese_roberta_L-12_H-768",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_

In [40]:
#metrics

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [41]:
training_args = transformers.TrainingArguments(
    output_dir="./chinese_roberta_ckpts",
    learning_rate=3e-5,#1e-4, 3e-4
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
trainer = transformers.Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_val,
    data_collator=data_collator,
    tokenizer = tokenizer,
)

In [43]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 107985
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 107985



ValueError: Expected input batch_size (7) to match target batch_size (1).

# SageMaker

In [21]:
import sagemaker

session = sagemaker.Session()

session_bucket = None
if session_bucket == None and session is not None:
    session_bucket = session.default_bucket()
    
role = sagemaker.get_execution_role()
session = sagemaker.Session(default_bucket = session_bucket)

print(f'role: {role}')
print(f'bucket: {session_bucket}')
print(f'session region: {session.boto_region_name}')

role: arn:aws:iam::635837196364:role/service-role/AmazonSageMaker-ExecutionRole-20220511T210233
bucket: sagemaker-us-east-1-635837196364
session region: us-east-1


Upload to S3 bucket

In [23]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()
s3_prefix = 'dataset/danmu_main'

#save data to S3
train_input_path = f's3://{session.default_bucket()}/{s3_prefix}/train'
#train_dataset.save_to_disk(train_input_path, fs = s3)
session.upload_data(path='data_csv/train.csv', bucket=session_bucket, key_prefix=s3_prefix+'/train')

test_input_path = f's3://{session.default_bucket()}/{s3_prefix}/test'
#test_dataset.save_to_disk(test_input_path, fs = s3)
session.upload_data(path='data_csv/test.csv', bucket=session_bucket, key_prefix=s3_prefix+'/test')

val_input_path = f's3://{session.default_bucket()}/{s3_prefix}/val'
session.upload_data(path='data_csv/val.csv', bucket=session_bucket, key_prefix=s3_prefix+'/val')


's3://sagemaker-us-east-1-635837196364/dataset/danmu_main/val/val.csv'

In [27]:
import sagemaker
from sagemaker.huggingface import HuggingFace

# gets role for executing training job

hyperparameters = {
    'model_name_or_path':'uer/chinese_roberta_L-12_H-768',
    'output_dir':'/opt/ml/model/chinese_roberta',
    # add your remaining hyperparameters 
    # more info here https://github.com/huggingface/transformers/tree/v4.17.0/examples/pytorch/text-classification
    'max_seq_length':128,
    'per_device_train_batch_size' : 64,
    'learning_rate' : 2e-5,
    'num_train_epochs': 5,
    #train
    'do_train': True,
    #data
    'train_file': '/opt/ml/input/data/train/train.csv',
    #'test_file': '/opt/ml/input/data/train/test.csv',
    'validation_file': '/opt/ml/input/data/val/val.csv',
}


In [28]:
# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.17.0'}

# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
    #entry_point='run_glue_t5.py',
    entry_point='run_glue.py',
    #source_dir='./examples/pytorch/text-classification',
    source_dir = './scripts',
    instance_type='ml.g5.xlarge',
    instance_count=1,
    role=role,
    it_config=git_config,
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    hyperparameters = hyperparameters
)


In [ ]:

# starting the train job
huggingface_estimator.fit({'train': train_input_path, 'test': test_input_path, 'val':val_input_path})

2022-05-12 02:25:16 Starting - Starting the training job...
2022-05-12 02:25:46 Starting - Preparing the instances for trainingProfilerReport-1652322316: InProgress
.........
2022-05-12 02:27:03 Downloading - Downloading input data...
2022-05-12 02:27:44 Training - Downloading the training image........................
2022-05-12 02:31:44 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-12 02:31:38,011 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-12 02:31:38,030 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-12 02:31:38,035 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-12 02:31:38,527 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channe

# Eval

### create predictor

In [12]:
import transformers
from sagemaker.huggingface import HuggingFace
from sagemaker.huggingface.model import HuggingFaceModel

import_from_s3 = False

if import_from_s3:
    #To restore old trainning job
    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        model_data="s3://sagemaker-us-east-1-635837196364/huggingface-pytorch-training-2022-05-12-00-16-12-039/output/model.tar.gz",  # path to your trained SageMaker model
        role=role,                                            # IAM role with permissions to create an endpoint
        transformers_version='4.17.0',                        # Transformers version used
        pytorch_version='1.10.2',                             # PyTorch version used
        py_version='py38',                                # Python version used
        env = {
            'HF_MODEL_ID':'uer/chinese_roberta_L-12_H-768',
            'HF_TASK': 'text-classification',

        },
    )
    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
       initial_instance_count=1,
       instance_type="ml.m5.4xlarge"
    )
else:
    predictor = huggingface_estimator.deploy(1, "ml.m5.4xlarge")

In [20]:
input = {"inputs": ['哈哈哈','跳舞']}

predictor.predict(input)

[{'label': 'LABEL_0', 'score': 0.5373631715774536},
 {'label': 'LABEL_0', 'score': 0.5193865895271301}]

In [31]:
#predictor.delete_endpoint()